In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers


dataset_path = keras.utils.get_file("SPY.csv", "http://localhost:8000/data/daily/SPY.csv")

column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
raw_dataset = pd.read_csv(dataset_path, 
#                           names=column_names, 
                          dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                          header=0,
                          na_values = "?", 
                          comment='\t',
                          sep=",",
                          skipinitialspace=True)


dataset = raw_dataset.copy()
dataset = dataset.sort_values(by=['Date'],ascending=False)

print len(dataset)

dataset.head(5)

dataset_stats = dataset.describe()
dataset_stats = dataset_stats.transpose()


# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old

# Simplification - If positive return, 1, else 0
def convert_labels_to_category(labels): 
    return map(lambda arr: 1 if arr[0] > 1 else 0, labels)

def convert_to_train(raw_dataset):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(5, len(dataset) - 50):

        feature_dataset = dataset[i:i+50].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            #dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
            #dataset['Close'].iloc[i-3] / latest_close, # 3 day trade
            dataset['Close'].iloc[i-5] / latest_close, # 5 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]
converted_feature_set = convert_to_train(dataset)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(converted_feature_set[0][0])]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer='sgd',
                metrics=['accuracy',precision, recall])
  return model

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model = build_model()
model.summary()



6445
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 64)                3264      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 7,489
Trainable params: 7,489
Non-trainable params: 0
_________________________________________________________________


In [15]:

#len(converted_feature_set[0][0])
train_data = np.array(converted_feature_set[0][900:])
train_labels = np.array(converted_feature_set[1][900:])

test_data = np.array(converted_feature_set[0][:900])
test_labels = np.array(converted_feature_set[1][:900])


In [16]:
print train_data
print train_labels

[[ 0.         -0.01299094 -0.02653142 ... -0.0270311  -0.02113519
  -0.02048569]
 [ 0.         -0.01336683 -0.01099929 ... -0.0080398  -0.00739864
  -0.007152  ]
 [ 0.          0.00233632 -0.00316383 ...  0.00588947  0.00613285
   0.00725232]
 ...
 [ 0.          0.         -0.00139082 ... -0.0006943  -0.00139082
   0.00278164]
 [ 0.         -0.00139082  0.00069652 ... -0.00139082  0.00278164
   0.01321391]
 [ 0.          0.00208444  0.01597333 ...  0.00416667  0.01458444
   0.01666667]]
[1 1 0 ... 0 0 0]


In [17]:

history = model.fit(
    train_data, train_labels,
    epochs=90, validation_split = 0.2, verbose=1,
    callbacks=[tensorboard]
)

Train on 4392 samples, validate on 1098 samples
Epoch 1/90


FailedPreconditionError: Attempting to use uninitialized value metrics_3/precision/precision/true_positives/count
	 [[{{node metrics_3/precision/precision/true_positives/count/read}} = Identity[T=DT_FLOAT, _class=["loc:@metrics_3/precision/precision/true_positives/AssignAdd"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](metrics_3/precision/precision/true_positives/count)]]